In [ ]:
test_train_x = []
test_train_y = []

results_iter = {}

# set hyper parameters

steps = 300
overlap = 50

epochs=300
batch_size=128
counter = 0;

#create dataset through sliding window
        
for key in data:
    for drive in data[key]:
        x_windows, y_windows = split_sequences(data[key][drive].values, steps, key, overlap)
        test_train_x.extend(x_windows)
        test_train_y.extend(y_windows)
        
y = array(test_train_y)
x = array(test_train_x)

#normalise data
        
scaler = MinMaxScaler(feature_range=(0, 1))

for data_x in x:
    scaler.partial_fit(data_x)

scaled_x = []
        
# Second pass
for data_x in x:
    scaled_x.append(list(map(list, zip(*scaler.transform(data_x)))))
    
#create five folds for training
    
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=22)
sss.get_n_splits(array(scaled_x), array(y))

#train and test on each of the five folds
                 
for train_index, test_index in sss.split(array(scaled_x), array(y)):
    x_train, x_test = array(scaled_x)[train_index], array(scaled_x)[test_index]
    y_train_numbers, y_test_numbers = array(y)[train_index], array(y)[test_index]
    
    #set more hyper parameters and model checkpoints
    
    learning_rate= 1e-3
    monitor='loss'
    optimization_mode='auto'
    compile_model=True
    factor = 1. / np.cbrt(2)
    
    #location to save model weights
    
    weight_fn = "./weights/%s_weights.h5" % counter
    
    #reducing learning rate on a loss plateau
    
    reduce_lr = ReduceLROnPlateau(monitor=monitor, patience=100, mode=optimization_mode,
                                  factor=factor, cooldown=0, min_lr=1e-4, verbose=2)
    
    #save model weights on checkpoint if the loss is the minimal seen so far in all training epochs
    
    model_checkpoint = ModelCheckpoint(weight_fn, verbose=0, mode=optimization_mode,
                                       monitor=monitor, save_best_only=True, save_weights_only=True)
    callback_list = [reduce_lr, model_checkpoint]

    optm = Adam(lr=learning_rate)
    
    #generate model
        
    model = generate_model()
    model.compile(optimizer=optm, loss='categorical_crossentropy', metrics=['accuracy'])
        
    classes = np.unique(y_train_numbers)
    le = LabelEncoder()
    y_ind = le.fit_transform(y_train_numbers.ravel())
    recip_freq = len(y_train_numbers) / (len(le.classes_) * np.bincount(y_ind).astype(np.float64))
    class_weight = recip_freq[le.transform(classes)]
        
    y_train = to_categorical(y_train_numbers, len(np.unique(y_train_numbers)))
    y_test = to_categorical(y_test_numbers, len(np.unique(y_test_numbers)))
    
    #train model

    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                class_weight=class_weight, verbose=0, callbacks=callback_list, validation_data=(x_test, y_test))
    
    #finished training, reload best weights
    
    model.load_weights(weight_fn)
    
    #time how long for classification
    
    start_time = time.time()
    
    #predict on test set
    
    y_pred = model.predict(x_test, batch_size=64, verbose=1)
    
    end_time = time.time()
    
    print('Total time: %s'%(end_time-start_time))
    
    #print the classification performance and save to array
    
    y_pred_bool = np.argmax(y_pred, axis=1)
    y_pred = to_categorical(y_pred_bool, len(np.unique(y_pred_bool)))
    print(classification_report(y_test, y_pred))
        
    results_iter[str(steps)+'_'+str(overlap)] = classification_report(y_test, y_pred)
    
    counter = counter + 1